# How To Use the Enviornment

In [0]:
import gym 
import random 
import numpy as np

In [0]:
#instantiate the enviornment
env_name = 'CartPole-v1'
env = gym.make(env_name)

In [0]:
#observation-space : what are the metrics/parameters on the basis of which represent the state
print(env.observation_space)#position velocity angular-velocity angle

print(env.action_space)

print(env.action_space.n)

Box(4,)
Discrete(2)
2


In [0]:
#it returns the initial state
state = env.reset();

for _ in range(200):
#     choose some action randomly
#     action = env.action_space.sample()
#     action_size = env.action_space.n
#     action = random.choice(range(action_size))
    
#     we will choose the action wisely if pole is moving right push the cart right and similarly for the other case
    pole_angle = state[2]
    #this is the policy on the basis of which we select the action
    action = 0 if pole_angle < 0 else 1
    #apply the action
    state,reward,done,info = env.step(action)
    
    env.render()#render means display
    
env.close()

/home/hp/anaconda3/lib/python3.5/site-packages/gym/logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [0]:
#another example where the action space is continious
env_name = 'MountainCarContinuous-v0'
# to create the new enviornment
env = gym.make(env_name)

In [0]:
print(env.observation_space)#two parameters
print(env.action_space)#only one action(continious)
print(env.action_space.low,env.action_space.high)#force applied to the car between -1 to 1
print(env.action_space.shape)#only one action passed at a time 

Box(2,)
Box(1,)
[-1.] [1.]
(1,)


In [0]:
state = env.reset()
for _ in range(200):
    action = env.action_space.sample()#choose a value randomly between -1 to 1
    state,reward,done,info = env.step(action)
    env.render()
env.close()

# Reinforcment Learning(Q-Learning)

In [0]:
#reinforcment learning lies between supervised and unsupervised we don't tell which action to take(whether to go left or right) but tell that a sequence of actions(left right left) was a good path or not

#q-learning we are going to maintain a table(q-table)

#frozen-lake state is the cell where the agent currently is,actions are up,down,left,right
#in case of 4*4 matrix  we have 16 states
#in q-table we maintain all the states(16) and for each state we maintain the reward for all the actions(4) and we choose the action which provides the maximum reward in a state

#our task is to fill the q-table
#we have to find how much total reward we will get if we take a action(go down by that path)

#we have to optimise the total-reward not the immediate-reward only
#you have the immediate reward plus the reward for the rest of the journey till the end(there is uncertainity as we are looking into the future now)

#the impact of the reward will reduce the more we go into the future(beacuse the uncertainity of recieving the reward increases)
#this  is called discount the reward 

#the aim of q-learning is to maximise the total  expected reward

# Q(st,at) =  rt+1 + G.Qmax(st+1)

In [0]:
import gym
import numpy as np
import random
import time

from IPython.display import clear_output

In [0]:
from gym.envs.registration import register
#to make the game non slippery
try:
    register(
        id='FrozenLake-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name' : '4x4','is_slippery':False},
        max_episode_steps=100,
        reward_threshold=0.78, # optimum = .8196
    )
except:
    pass

env_name = 'FrozenLake-v0'
env = gym.make(env_name)

In [0]:
print(env.observation_space)
print(env.action_space)

type(env.action_space)

Discrete(16)
Discrete(4)


gym.spaces.discrete.Discrete

In [0]:
#the agent should work for both discrete and continious games

In [0]:
#we define an agent class to create an agent
class Agent():
    #constructor
    def __init__(self,env):
        self.is_discrete = type(env.action_space) == gym.spaces.discrete.Discrete
        if self.is_discrete:
            self.action_size = env.action_space.n
        else:
            self.action_low = env.action_space.low
            self.action_high = env.action_space.high
            self.action_shape = env.action_space.shape
        
    #we will call this function to figure out which action should we take next
    #so all the logic will be in this function
    def get_action(self,state):
        # action = env.action_space.sample()
        if self.is_discrete:
            action = random.choice(range(self.action_size))
        else:
            action = np.random.uniform(self.action_low,self.action_high,self.action_shape)
        return action   

In [0]:
agent = Agent(env)

In [0]:
#everytime you play a game is called an episode    
for ep in range(2):
    #initial state
    state = env.reset()#0
    done = False
    while not done:
        action = agent.get_action(state)
        next_state,reward,done,info = env.step(action)
        env.render()#we dont have to close as it will render on the screen itself
        time.sleep(0.05)
        clear_output(wait=True)

  (Down)
SFFF
FHFH
FFFH
HFFG


In [0]:
#class QAgent inherits Agent class
#learning rate is kept small to avoid fluctuations
class QAgent(Agent):
    def __init__(self,env,discount_rate=0.97,learning_rate=0.01):
        super().__init__(env)
        self.state_size = env.observation_space.n
        #most of the times we will choose the action with the max q-value to maximise the reward
        #but in some cases we will choose a random action to explore a better path that may be available
        self.exploration_rate = 1.0 #epsilon
        #at start the exploration rate is 1 because all the q values are random
        #after every episode exploration rate will be reduced, as q-values start becoming meaningful
        #as we are learning q-value we will reduce over tendency to explore
        self.discount_rate = discount_rate #gamma
        self.learning_rate  =learning_rate #alpha
        self.q_table = 1e-4 * np.random.random([self.state_size,self.action_size])
        
    def get_action(self,state):
        q_state = self.q_table[state]
        #action chosen on basis of q_value
        action_greedy = np.argmax(q_state)
        #action chosen randomly
        action_random = super().get_action(state)
        #need to choose between these two actions
        return action_random if random.random()<self.exploration_rate else action_greedy
    
    def train(self,experience):
        state,action,next_state,reward,done = experience
        
        q_next = self.q_table[next_state]
    
        #if you have reached the goal state then there is no expected reward in future
        q_next = np.zeros([self.action_size]) if done else q_next
        
        # total_reward = immediate_reward + expected_reward in future
        q_target = reward + self.discount_rate * np.max(q_next)
        
        q_update = q_target - self.q_table[state,action] #expected reward calculated - orignal value in q_table
        
        self.q_table[state,action] += self.learning_rate*q_update
        
        #when one episode is completed we reduce our tendecy to explore
        if done:
            self.exploration_rate *= 0.99

In [0]:
agent = QAgent(env)

In [0]:
total_reward = 0
for ep in range(100):
    #initial state
    state = env.reset()#0
    done = False
    while not done:
        action = agent.get_action(state)
        next_state,reward,done,info = env.step(action)
        agent.train((state,action,next_state,reward,done))
        state = next_state
        total_reward += reward
        print("s: ",state," a: ",action)
        print("Episode: ",ep," Reward: ",total_reward," Explore: ",agent.exploration_rate)
        
        env.render()
        
        print(agent.q_table)
        
        time.sleep(0.05)
        clear_output(wait=True)

s:  12  a:  1
Episode:  99  Reward:  1.0  Explore:  0.358748297681892
  (Down)
SFFF
FHFH
FFFH
HFFG
[[3.68661318e-05 6.75374788e-05 6.74436098e-05 6.47374251e-05]
 [4.75017310e-05 2.24187138e-05 2.59189751e-05 2.20851530e-05]
 [6.93681163e-05 2.67686178e-05 1.23655433e-05 1.52886390e-05]
 [1.85738434e-05 5.12426286e-05 7.24489014e-05 6.01941592e-05]
 [1.77149480e-05 4.59608759e-05 6.77874757e-05 2.12447639e-05]
 [6.07175591e-06 1.65637967e-05 9.10221060e-05 2.70826131e-06]
 [3.51067111e-05 1.68051713e-05 4.78199258e-05 7.94139641e-05]
 [3.07400426e-05 8.32873674e-05 7.55470320e-05 7.75336445e-05]
 [5.87015244e-05 7.58361437e-05 8.34480507e-05 7.87326332e-05]
 [6.10815839e-05 1.89020577e-05 1.00252699e-05 3.34534796e-05]
 [8.10621473e-05 2.35744603e-05 5.21444799e-05 7.29577279e-05]
 [2.15004943e-05 3.51842748e-05 8.19211404e-05 7.21198541e-05]
 [8.44808018e-05 5.23584787e-05 7.24382893e-05 9.96305006e-05]
 [7.65358137e-05 1.31739669e-05 3.35608191e-05 7.41795469e-05]
 [6.45046588e-05 1.

In [0]:
#in a good training algorithm you should give it a negative reward for each step to reduce the number of steps to reach the goal state

# Using Neural Networks

last time we did q-learning manually(updated the qtable manually)
now we will implement q-learning using neural-networks
so the q-values will become weights and neural network will optimise them
earlier in each iteration only one value was getting updated but now multiple values will get updated so it will be faster
and further optimisation can be done by multiple times training on previous experiences called experience relay

In [0]:
import tensorflow as tf

In [0]:
#the input to the neural network will be the state and the output will be the action

In [0]:
class QNAgent(Agent):
    def __init__(self,env,discount_rate=0.97,learning_rate=0.01):
        super().__init__(env)
        self.state_size = env.observation_space.n
        self.exploration_rate = 1.0
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        
        self.build_model()
        
        #we have to start running the model after we have created it
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        
        
    def build_model(self):
        
        #so that we can assign name to layers in tf
        tf.reset_default_graph()
        
        #declaring the type of variables
        self.state_in = tf.placeholder(tf.int32,shape=[1])
        self.action_in = tf.placeholder(tf.int32,shape=[1])
        self.target_in = tf.placeholder(tf.float32,shape=[1])
        
        #one hot encoding 
        self.state = tf.one_hot(self.state_in,depth=self.state_size)#16
        self.action = tf.one_hot(self.action_in,depth=self.action_size)#4
        
        #a dense layer with units = number_of_actions
        #the result of the dense layer will be a row of the q-table
        #output of the layer is one-hot encoded
        self.q_state = tf.layers.dense(self.state,units=self.action_size,name='q_table')
        
        #then we multiply the output of the dense layer with the actual action (vector of size 4)
        #q_action will be a scalar because we are doing reduce_sum on a vector of size 4
        self.q_action = tf.reduce_sum(tf.multiply(self.q_state,self.action),axis=1)
        
        
        #squaring and adding the errors
        self.loss = tf.reduce_sum(tf.square(self.target_in-self.q_action))
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)
        
    def get_action(self,state):
        #in this no backprop is there only one feed forward(prediction)
        #we give a state as input and ask for predicted q-values for that state
        q_state = self.sess.run(self.q_state,feed_dict={self.state_in:[state]})
        action_greedy = np.argmax(q_state)
        action_random = super().get_action(random)
        return action_random if random.random()<self.exploration_rate else action_greedy
    
    #experience is a tuple with multiple values
    def train(self,experience):
        #list comphrension all the values on lhs are list
        state,action,next_state,reward,done =  [[exp]for exp in experience]
        
        #why we are doing this???? essentially we just need to give the current_state as input and get the action as output from dense layer
        #but also we need to realise how are we getting the q_value for the current_state
        #current_state reward + discount * max q_value of next state
        #so we need to get the q-values for the next state but now we dont have a table
        #those q-values are in the dense layer and this is how we get them
        #no training i.e backprop in this case as well
        q_next = self.sess.run(self.q_state,feed_dict={self.state_in:next_state})
        
        #edge case next state wont exist if the game is over
        #q_next = np.zeros([self.action_size]) if done else q_next
        q_next[done] = np.zeros([self.action_size])
        
        q_target = reward + self.discount_rate * np.max(q_next)
        
        #training is done in this case the above two were only predictions
        #for a given state if i take a action what is the predicted q-value and i also have the target q-value
        feed = {self.state_in:state,self.action_in:action,self.target_in:q_target}
        self.sess.run(self.optimizer,feed_dict=feed)
        
        #after every episode reduce your exploration_rate
        if(done[0]):
            #because done is a list now
            self.exploration_rate *= 0.99
          
    #destructor 
    def __del__(self):
        self.sess.close()

In [6]:
agent = QNAgent(env)

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [7]:
total_reward = 0.0
for ep in range(100):
    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state)
        next_state,reward,done,info = env.step(action)
        agent.train((state,action,next_state,reward,done))
        state = next_state
        total_reward += reward#in this game we get reward only when we reach the goal state
        print("s: ",state," a: ",action)
        print("Episode: ",ep," Reward: ",total_reward," Explore: ",agent.exploration_rate)
        env.render()
        # ReUse = True so if we run the cell again,previously learned weights in the q-table will be the start for the next iteration,i.e weights will not be reset in the next iteration when we run the cell again
        with tf.variable_scope('q_table',reuse = True):
            #the weights in the dense layer now represent our q-value
            weights = agent.sess.run(tf.get_variable('kernel'))
            print(weights)
        time.sleep(0.05)
        clear_output(wait=True)

s:  5  a:  3
Episode:  99  Reward:  2.0  Explore:  0.36603234127322926
  (Up)
SFFF
FHFH
FFFH
HFFG
[[ 0.04121775  0.11725765  0.10114238  0.03047478]
 [-0.07643432 -0.10581105 -0.13324344  0.02383582]
 [ 0.15284581  0.05639105  0.01886346  0.08038311]
 [ 0.03517211  0.10905992 -0.2304262   0.1074306 ]
 [ 0.11117578  0.13411562 -0.22252291 -0.0144773 ]
 [ 0.06013334  0.26583594  0.45679402  0.19534427]
 [ 0.17942011 -0.10360501 -0.11926575 -0.02127964]
 [-0.18365738  0.46836185 -0.15440989 -0.1180737 ]
 [-0.30276006 -0.07258799 -0.09086517  0.11256143]
 [-0.01686391  0.15828489  0.0325565  -0.04545236]
 [ 0.02242352  0.10428189  0.05802891  0.02220099]
 [ 0.19901901  0.04223073  0.16622734  0.28041744]
 [ 0.15605229 -0.23030052  0.0764249  -0.28423566]
 [-0.05185646 -0.3021306  -0.12070956 -0.32087773]
 [-0.09781185 -0.20094383  0.12630574  0.18454683]
 [ 0.03428936 -0.29362905 -0.0604226  -0.46508503]]


# QLearning + Neural Nets + Experience Replay

there are few paths that take you to the goal and a lot of paths that take you to the hole
since the paths to goal are less so time of training done upon those paths will be less
so a lot of training is done because of holes and not because of goals
if we give negative reward to the holes then it knows that it has to avoid the holes which is not in this case
so in order to maximise the reward, the rare paths upon which we are not able to go more number of times,we want to train upon those paths more

This will allow us to train on rare experiences more often.

1. Take a buffer. Keep adding new experiences into the buffer, and remove old ones.
2. Take a sample of experiences from the buffer to train on.

You will notice that we reach the Goal more often now.

In [9]:
from collections import deque
import random

print("Using OpenAI Gym:", gym.__version__)
print("Using Tensorflow:", tf.__version__)

Using OpenAI Gym: 0.10.11
Using Tensorflow: 1.13.1


In [0]:
class QNRAgent(Agent):
    def __init__(self, env, discount_rate=0.97, learning_rate=0.001): # Smaller learning rate
        super().__init__(env)
        self.state_size = env.observation_space.n
        
        self.eps = 1.0
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        self.build_model()
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        self.replay_buffer = deque(maxlen=1000) # 
        
    def build_model(self):
        tf.reset_default_graph()
        self.state_in = tf.placeholder(tf.int32, shape=[None]) # None means any shape
        self.action_in = tf.placeholder(tf.int32, shape=[None]) #
        self.target_in = tf.placeholder(tf.float32, shape=[None]) #
        
        self.state = tf.one_hot(self.state_in, depth=self.state_size)
        self.action = tf.one_hot(self.action_in, depth=self.action_size)
        
        self.q_state = tf.layers.dense(self.state, units=self.action_size, name="q_table")
        self.q_action = tf.reduce_sum(tf.multiply(self.q_state, self.action), axis=1)
        
        self.loss = tf.reduce_sum(tf.square(self.target_in - self.q_action))
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)
        
    def get_action(self, state):
        q_state = self.sess.run(self.q_state, feed_dict={self.state_in: [state]})
        action_greedy = np.argmax(q_state)
        action_random = super().get_action(state)
        return action_random if random.random() < self.eps else action_greedy
    
    def train(self, experience, batch_size=50):
        self.replay_buffer.append(experience) # 
        samples = random.choices(self.replay_buffer) 
        state, action, next_state, reward, done = (list(col) for col in zip(experience, *samples)) # 
        # state, action, next_state, reward, done = ([exp] for exp in experience)
        
        q_next = self.sess.run(self.q_state, feed_dict={self.state_in: next_state})
        q_next[done] = np.zeros([self.action_size])
        q_target = reward + self.discount_rate * np.max(q_next, axis=1)
        
        feed = {self.state_in: state, self.action_in: action, self.target_in: q_target}
        self.sess.run(self.optimizer, feed_dict=feed)
        
        if experience[4]:
            self.eps = self.eps * 0.99
            
    def __del__(self):
        self.sess.close()
        
agent = QNRAgent(env)

In [14]:
total_reward = 0
for ep in range(100):
    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        agent.train((state,action,next_state,reward,done))
        state = next_state
        total_reward += reward
        
        print("s:", state, "a:", action)
        print("Episode: {}, Total reward: {}, eps: {}".format(ep,total_reward,agent.eps))
        env.render()
        
        # Reuse=True... So if we run again, previously learned weights in q-table will be the starting point.
        with tf.variable_scope("q_table", reuse=True):
            weights = agent.sess.run(tf.get_variable("kernel"))
            print(weights)
        time.sleep(0.05)
        clear_output(wait=True)

s: 5 a: 3
Episode: 99, Total reward: 9.0, eps: 0.13397967485796175
  (Up)
SFFF
FHFH
FFFH
HFFG
[[ 0.0067195   0.17599882  0.09380232  0.2239123 ]
 [ 0.01076017  0.13619302 -0.05322899  0.21017969]
 [ 0.0172868  -0.02394445  0.07531707  0.20565414]
 [-0.27536845  0.14818554 -0.35104477  0.21138243]
 [ 0.05686286  0.17754386  0.10101318  0.25296536]
 [ 0.35862166 -0.11615252  0.3615265  -0.26941258]
 [-0.15395798  0.16146454  0.04263688  0.0222161 ]
 [-0.42393816 -0.44041643  0.38130927  0.20673525]
 [-0.27182758  0.24018717 -0.17501223  0.28538427]
 [ 0.09451574  0.2660884  -0.04190182  0.24824321]
 [ 0.00087914  0.25509307  0.17645127  0.00224096]
 [ 0.10008883  0.04607654  0.11425209 -0.24230176]
 [ 0.4153232  -0.30978847 -0.0752629  -0.28703558]
 [-0.12627564  0.14441176 -0.21078219  0.08545002]
 [ 0.21787913  0.06302729  0.4139699   0.45066664]
 [-0.12409168 -0.45314607 -0.49180356 -0.53413004]]
